In [55]:
import streamlit as st
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

from langchain.globals import set_verbose
import utils.chains_lcel as chains
from utils.sidebar import sidebar
import utils.llm_models as llms

from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.messages.utils import convert_to_openai_messages, convert_to_messages

In [68]:
"""
Tools and Agent implementation for the Virtual TA system.
"""

from langchain.tools import StructuredTool
from typing import Optional, List
from langchain.schema.language_model import BaseLanguageModel
from pydantic import BaseModel, Field
import streamlit as st

# Tool argument schemas
class RagArgs(BaseModel):
    query: str = Field(..., description="The query to get course information for")

class ExplainArgs(BaseModel):
    query: str = Field(..., description="The query to explain")
    # previous_query: Optional[str] = Field(default="", description="Previous query for context")

class ExerciseArgs(BaseModel):
    query: str = Field(..., description="The query to generate exercise for")
    previous_query: str = Field(default="", description="Previous query for context")
    skill_level: str = Field(default="beginner", description="Student's skill level")
    previous_exercise: str = Field(default="", description="Previous exercise for context")

class AnalyticsArgs(BaseModel):
    query: str = Field(..., description="The query about data analytics")
    chat_history: str = Field(default="", description="Previous chat history for context")

class DebugArgs(BaseModel):
    query: str = Field(..., description="The code or error to debug")
    chat_history: str = Field(default="", description="Previous chat history for context")

class ChatArgs(BaseModel):
    query: str = Field(..., description="The message to respond to")
    

# Tool definitions
def rag_tool(chain):
    """Tool for retrieving course-related information."""
    def _run(args: RagArgs) -> str:
        return chain.stream(args.query)
    
    return StructuredTool(
        name="course_information",
        description="Use this tool for course-specific information, ex: instructor, syllabus, policies or assignments.",
        func=_run,
        args_schema=RagArgs
    )

def explain_tool(chain):
    """Tool for explaining technical concepts."""
    def _run(args: ExplainArgs) -> str:
        return chain.invoke({"query": args.query, "chat_history": args.previous_query})
    
    return StructuredTool(
        name="explain_concept",
        description="Use this tool when the query asks for explanation of Python, SQL, or other concepts.",
        func=_run,
        args_schema=ExplainArgs
    )

def exercise_tool(chain):
    """Tool for generating exercise questions."""
    def _run(args: ExerciseArgs) -> str:
        return chain.invoke({
            "current_query": args.query,
            "previous_query": args.previous_query,
            "skill_level": args.skill_level,
            "previous_exercise": args.previous_exercise
        })
    
    return StructuredTool(
        name="generate_exercise",
        description="Use this tool when the query asks for Python or SQL practice exercises.",
        func=_run,
        args_schema=ExerciseArgs
    )

def analytics_tool(chain):
    """Tool for data analytics explanations."""
    def _run(args: AnalyticsArgs) -> str:
        return chain.invoke({
            "query": args.query,
            "chat_history": args.chat_history
        })
    
    return StructuredTool(
        name="explain_analytics",
        description="Use this tool when the query is about data analysis with pandas, matplotlib, seaborn, or statistics.",
        func=_run,
        args_schema=AnalyticsArgs
    )

def debug_tool(chain):
    """Tool for debugging assistance."""
    def _run(args: DebugArgs) -> str:
        return chain.invoke({
            "query": args.query,
            "chat_history": args.chat_history
        })
    
    return StructuredTool(
        name="debug_code",
        description="Use this tool when the query is about code errors or debugging help.",
        func=_run,
        args_schema=DebugArgs
    )

def chat_tool(chain):
    """Tool for general conversation."""
    def _run(args, chat_history) -> str:
        
        print("--"*10, "inside the chat tool")
        print(args.query)
        return chain.stream({
            "query": args,
            "chat_history": chat_history
        })
    
    return StructuredTool(
        name="general_chat",
        description="Use this tool for general conversation or when no other tools are appropriate.",
        func=_run,
        args_schema=ChatArgs
    )

def create_tool_chain(llm: BaseLanguageModel, chain_dict: dict):
    """Create a tool-enabled LLM chain."""
    tools = [
        # rag_tool(chain_dict['rag']),
        explain_tool(chain_dict['explain']),
        # exercise_tool(chain_dict['exercise']),
        # analytics_tool(chain_dict['analytics']),
        debug_tool(chain_dict['debug']),
        chat_tool(chain_dict['chat'])
    ]
    
    # Bind tools to LLM with system message
    return llm.bind_tools(tools, tool_choice="any")



In [67]:
output_parser = StrOutputParser()

def chat_chain(llm):
    messages = [
        ("system", """You are a virtual teaching assistant for an Applied data analytics with Python class. Converse with the student in a friendly and engaging manner, considering the chat history. Your response should be concise and relevant to the student's query. Limit your response to 100 tokens."""),
        MessagesPlaceholder("chat_history"),
        ("human", "{query}")
    ]

    prompt = ChatPromptTemplate.from_messages(messages)

    # setup = RunnableParallel(
    #         {"query": RunnablePassthrough(),
    #          "chat_history": RunnablePassthrough()
    #          }
    #     )
        
    # chain = setup | prompt | llm | output_parser
    chain = prompt | llm | output_parser

    return chain
# Setup LLM models
gpt4o_mini = llms.openai_gpt4o_mini
gpt4o_mini_json = llms.openai_4o_mini_json
claude_sonnet = llms.claude_sonnet
claude_haiku = llms.claude_haiku
gpt4o = llms.openai_gpt4o

# Create base chains
chains_dict = {
    # 'rag': chains.rag_chain(claude_haiku, retriever),
    # 'exercise': chains.exercise_chain(claude_sonnet),
    'chat': chat_chain(gpt4o_mini),
    # 'chat': chains.chat_chain(gpt4o_mini),
    'explain': chains.code_chain(claude_sonnet),
    'debug': chains.code_chain(claude_haiku),
}

# Create tool chain
tool_agent = create_tool_chain(gpt4o_mini, chains_dict)

In [4]:
user_query = "hi i'm lili"

tool_call = tool_agent.invoke(
            f"""
            You're a helpful tool callling agent Your only task is to decide which tool to call based on the user query and chat history, and generate the appropriate arguements for the tool call.
            
            Query: {user_query}\n
            """)
print(tool_call.tool_calls)

[{'name': 'general_chat', 'args': {'query': "hi i'm lili"}, 'id': 'call_Tn5SUZvv95wm9vt8frJa520x', 'type': 'tool_call'}]


In [5]:
name=tool_call.tool_calls[0]["name"]
args=tool_call.tool_calls[0]['args']

print(name)
print(args)

general_chat
{'query': "hi i'm lili"}


In [8]:
chains.chat_chain(gpt4o_mini).invoke(input={
    
    "query": user_query,
    "chat_history":
    [("ai", "hi, how can i help you?"),
     
     ]
    }
)

"Hi Lili! Great to meet you! How's your journey in applied data analytics with Python going so far?"

In [57]:
def convert_dict_to_tuples(messages):
    chat_history = []
    for message in messages:
        if message['role'] == 'assistant':
            chat_history.append(("assistant", message['content']))
        elif message['role'] == 'user':
            chat_history.append(("human", message['content']))  # note: 'user' becomes 'human'
    return chat_history

In [78]:


chat_history = [
    AIMessage(content="what's your name?"),
    HumanMessage(content="my name is lili"),
    AIMessage(content="hi lili, how can i help you?"),
    HumanMessage(content="can you explain what a function is in python?"),
    AIMessage(content="a function is a block of code that performs a specific task. it can take inputs, process them, and return an output. functions help organize code and make it reusable."),

    ]

user_query = "give me an code example highlighting return value with business context"

chain = chat_chain(gpt4o_mini)

response1 = chains_dict['chat'].invoke(
    {
        'query': user_query, 
    "chat_history": chat_history
    # convert_to_openai_messages(chat_history)
    }
)
print(response1)


# response2 = chat_tool(chain).invoke(
#     input={
#         'args': ChatArgs(user_query) 
#     # "chat_history": chat_history
#     }
# )



# response = chat_tool(chain).func(
#     args=ChatArgs(query=user_query),
#     chat_history=chat_history
# )
# for chunk in response:
#     print(chunk, end='')

sure! here's a simple example:

```python
def calculate_discount(price, discount_rate):
    discount = price * discount_rate
    return price - discount

final_price = calculate_discount(100, 0.2)
print(f"The final price after discount is: ${final_price}")
```

in this example, the function calculates the final price after applying a discount, which is useful in a retail business context.


In [79]:
args={"query": "can you explain what a function is in python?",}

{
                'chat_history': chat_history,
                **args}

{'chat_history': [AIMessage(content="what's your name?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='my name is lili', additional_kwargs={}, response_metadata={}),
  AIMessage(content='hi lili, how can i help you?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='can you explain what a function is in python?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='a function is a block of code that performs a specific task. it can take inputs, process them, and return an output. functions help organize code and make it reusable.', additional_kwargs={}, response_metadata={})],
 'query': 'can you explain what a function is in python?'}